# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f54c05eaa90>
├── 'a' --> tensor([[ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873]])
└── 'x' --> <FastTreeValue 0x7f54c05eac40>
    └── 'c' --> tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                        [-0.9451,  1.3151, -1.0868, -0.0230],
                        [-2.4932,  1.5259, -1.2255,  1.0099]])

In [4]:
t.a

tensor([[ 0.3830,  0.0220,  0.7479],
        [ 0.4820, -0.9826, -0.1873]])

In [5]:
%timeit t.a

62.3 ns ± 0.0191 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f54c05eaa90>
├── 'a' --> tensor([[ 0.9871,  0.0186, -1.1179],
│                   [-0.9618,  0.2985, -1.7517]])
└── 'x' --> <FastTreeValue 0x7f54c05eac40>
    └── 'c' --> tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                        [-0.9451,  1.3151, -1.0868, -0.0230],
                        [-2.4932,  1.5259, -1.2255,  1.0099]])

In [7]:
%timeit t.a = new_value

62.9 ns ± 0.0117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873]]),
    x: Batch(
           c: tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                      [-0.9451,  1.3151, -1.0868, -0.0230],
                      [-2.4932,  1.5259, -1.2255,  1.0099]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3830,  0.0220,  0.7479],
        [ 0.4820, -0.9826, -0.1873]])

In [11]:
%timeit b.a

51.4 ns ± 0.0495 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.1168,  1.0119,  2.3696],
               [ 0.3685,  0.8064, -2.2964]]),
    x: Batch(
           c: tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                      [-0.9451,  1.3151, -1.0868, -0.0230],
                      [-2.4932,  1.5259, -1.2255,  1.0099]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.273 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.166 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 441 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f54c0604490>
├── 'a' --> tensor([[[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]],
│           
│                   [[ 0.3830,  0.0220,  0.7479],
│                    [ 0.4820, -0.9826, -0.1873]]])
└── 'x' --> <FastTreeValue 0x7f54c0604a60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f54143499d0>
├── 'a' --> tensor([[ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873],
│                   [ 0.3830,  0.0220,  0.7479],
│                   [ 0.4820, -0.9826, -0.1873]])
└── 'x' --> <FastTreeValue 0x7f5414349910>
    └── 'c' --> tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                        [-0.9451,  1.3151, -1.0868, -0.0230],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 31.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 58.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]],
       
               [[ 0.3830,  0.0220,  0.7479],
                [ 0.4820, -0.9826, -0.1873]]]),
    x: Batch(
           c: tensor([[[ 0.0574,  1.1141,  1.6522, -0.2473],
                       [-0.9451,  1.3151, -1.0868, -0.0230],
                       [-2.4932,  1.5259, -1.2255,  1.0099]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873],
               [ 0.3830,  0.0220,  0.7479],
               [ 0.4820, -0.9826, -0.1873]]),
    x: Batch(
           c: tensor([[ 0.0574,  1.1141,  1.6522, -0.2473],
                      [-0.9451,  1.3151, -1.0868, -0.0230],
                      [-2.4932,  1.5259, -1.2255,  1.0099],
                      [ 0.0574,  1.1141,  1.6522, -0.2473],
                      [-0.9451,  

In [28]:
%timeit Batch.cat(batches)

147 µs ± 564 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
